# In this notebook, we will apply data generator to monitor test outcome on Grafana dashboard.

In [7]:
# TensorFlow and tf.keras to load the data
import tensorflow as tf
from tensorflow import keras

# Commonly used modules
import numpy as np
import os
import sys

# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
#import cv2
import IPython
from six.moves import urllib
from keras.layers import Activation, Dense
print(tf.__version__)

2.2.0


# The data set is Boston Housing Prices classification data set https://www.kaggle.com/vikrishnan/boston-house-prices
## Step 1: Pre-process the data (normalization)

In [8]:
(train_features, train_labels), (test_features, test_labels) = keras.datasets.boston_housing.load_data()

# get per-feature statistics (mean, standard deviation) from the training set to normalize by
train_mean = np.mean(train_features, axis=0)
train_std = np.std(train_features, axis=0)
train_features = (train_features - train_mean) / train_std
print("Number of features per sample=", np.shape(train_features)[1])

Number of features per sample= 13


In [9]:
# Do the same for test features
test_mean = np.mean(test_features, axis=0)
test_std = np.std(test_features, axis=0)
test_features = (test_features - test_mean) / test_std

## Step 2: Define a simple Neural network model

In [10]:
def nn_model():
    model = keras.Sequential([
        Dense(100, activation=tf.nn.relu, input_shape=[len(train_features[0])]),
        Dense(1)
    ])

    model.compile(optimizer=tf.optimizers.Adam(), 
                  loss='mse',
                  metrics=['mae', 'mse'])
    return model

## Next, lets train the model

In [11]:
#Now, lets train the model
model = nn_model()

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
history = model.fit(train_features, train_labels, epochs=1000, verbose=1, validation_split = 0.1,
                    callbacks=[early_stop])

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch


Epoch 1/1000
12/12 [==============================] - 0s 14ms/step - loss: 570.8159 - mae: 21.9485 - mse: 570.8159 - val_loss: 469.7301 - val_mae: 20.6616 - val_mse: 469.7301
Epoch 2/1000
12/12 [==============================] - 0s 27ms/step - loss: 541.4741 - mae: 21.3402 - mse: 541.4741 - val_loss: 443.0724 - val_mae: 20.0429 - val_mse: 443.0724
Epoch 3/1000
12/12 [==============================] - 0s 4ms/step - loss: 512.5126 - mae: 20.7169 - mse: 512.5126 - val_loss: 415.6546 - val_mae: 19.3847 - val_mse: 415.6546
Epoch 4/1000
12/12 [==============================] - 0s 3ms/step - loss: 482.5405 - mae: 20.0467 - mse: 482.5405 - val_loss: 387.1421 - val_mae: 18.6637 - val_mse: 387.1421
Epoch 5/1000
12/12 [==============================] - 0s 4ms/step - loss: 450.8488 - mae: 19.3121 - mse: 450.8488 - val_loss: 357.3724 - val_mae: 17.8792 - val_mse: 357.3724
Epoch 6/1000
12/12 [==============================] - 0s 4ms/step - loss: 417.3324 - mae: 18.4946 - mse: 417.3324 - val_loss: 32

In [12]:
# show RMSE measure to compare to Kaggle leaderboard on https://www.kaggle.com/c/boston-housing/leaderboard
rmse_final = np.sqrt(float(hist['val_mse'].tail(1)))
print()
print('Final Root Mean Square Error on validation set: {}'.format(round(rmse_final, 3)))


Final Root Mean Square Error on validation set: 2.247


In [13]:
print(hist)

           loss        mae         mse    val_loss    val_mae     val_mse  \
0    570.815918  21.948477  570.815918  469.730133  20.661583  469.730133   
1    541.474060  21.340160  541.474060  443.072418  20.042877  443.072418   
2    512.512634  20.716883  512.512634  415.654572  19.384661  415.654572   
3    482.540466  20.046680  482.540466  387.142090  18.663713  387.142090   
4    450.848785  19.312140  450.848785  357.372437  17.879181  357.372437   
..          ...        ...         ...         ...        ...         ...   
429    5.262722   1.595615    5.262722    4.971745   1.823374    4.971745   
430    5.276659   1.609020    5.276659    5.208706   1.866811    5.208706   
431    5.266827   1.595687    5.266827    5.500435   1.909477    5.500435   
432    5.239838   1.576322    5.239838    5.298979   1.871805    5.298979   
433    5.232708   1.589226    5.232708    5.049713   1.827765    5.049714   

     epoch  
0        0  
1        1  
2        2  
3        3  
4        4

In [14]:
mse, _, _ = model.evaluate(test_features, test_labels)
rmse = np.sqrt(mse)
print('Root Mean Square Error on test set: {}'.format(round(rmse, 3)))

4/4 [==============================] - 0s 811us/step - loss: 15.3315 - mae: 2.6686 - mse: 15.3315
Root Mean Square Error on test set: 3.916


## Now, lets generate more test samples from test data and monitor them, as priduction data sets

In [15]:
print(np.shape(train_features), np.shape(train_labels))
print(np.shape(test_features), np.shape(test_labels))

(404, 13) (404,)
(102, 13) (102,)


In [16]:
num=np.shape(test_labels)[0]
import random

In [17]:
#Lets import the ml_monotor library
import ml_monitor
import time
import numpy as np

In [18]:
my_monitor = ml_monitor.Monitor()
my_monitor.start()

2022-08-06 14:15:24,815 [65263] INFO     ml_monitor.logging: Starting metrics logging thread...


In [19]:
epochs=100000
for i in range(epochs):
    indx=random.randint(0,num-1)
    x_batch=np.expand_dims(test_features[indx,:],axis=0)
    y_batch=np.expand_dims(test_labels[indx],axis=0)
    mse, _, _ = model.evaluate(x_batch, y_batch, verbose=0)
    my_monitor.monitor("n_rmse", np.sqrt(mse))

2022-08-06 14:15:29,819 [65263] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-08-06 14:15:34,830 [65263] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-08-06 14:15:39,838 [65263] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-08-06 14:15:44,901 [65263] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-08-06 14:15:49,906 [65263] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-08-06 14:15:54,913 [65263] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-08-06 14:15:59,918 [65263] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-08-06 14:16:04,925 [65263] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-08-06 14:16:09,929 [65263] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-08-06 14:16:14,938 [65263] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-08-06 14:16:19,944 [65263] INFO    

KeyboardInterrupt: 